In [ ]:
"""
 R O N A L D
+      A N D
+  N A N C Y
------------
 R E A G A N
"""

In [5]:
import pyomo.environ as pyo

In [59]:
# unique letters
letters = list(set([c for c in 'RONALDANDNANCYREAGEN']))
print('Set of letters')
print(letters)

# digits
digits = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
print("Set of digits")
print(digits)

Set of letters
['N', 'C', 'Y', 'G', 'A', 'R', 'E', 'D', 'L', 'O']
Set of digits
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [39]:
# Pyomo model


# create a model instance
m = pyo.ConcreteModel()

# decision variables

m.x = pyo.Var(letters, digits, within = pyo.Binary) # binary variable if letter c is assigned digit i
m.d = pyo.Var(letters, bounds = (0, 9), within = pyo.NonNegativeReals) # value of each letter
m.co1 = pyo.Var(bounds = (0, 3), within = pyo.NonNegativeIntegers) # carryover after first column addition
m.co2= pyo.Var(bounds = (0, 3), within = pyo.NonNegativeIntegers) # carryover after second column addition
m.co3 = pyo.Var(bounds = (0, 3), within = pyo.NonNegativeIntegers) # carryover after third column addition
m.co4 = pyo.Var(bounds = (0, 3), within = pyo.NonNegativeIntegers) # carryover after fourth column addition
m.co5 = pyo.Var(bounds = (0, 3), within = pyo.NonNegativeIntegers) # carryover after fifth column addition
m.y = pyo.Var(within = pyo.NonNegativeIntegers) # integer value when AND is divided by 3


In [40]:
# ensure that each letter is assigned exactly one digit
def assign_rule(m, c):
    return sum(m.x[c, i] for i in digits) == 1
m.assign_cons = pyo.Constraint(letters, rule = assign_rule)

# ensure each digit is assigned exactly one letter
def assign2_rule(m, i):
    return sum(m.x[c, i] for c in letters) == 1
m.assign2_cons = pyo.Constraint(digits, rule = assign2_rule)

# calculate the value of digit
def calcint_rule(m, c):
    return m.d[c] == sum(i * m.x[c, i] for i in digits)
m.calcint_cons = pyo.Constraint(letters, rule = calcint_rule)

In [41]:
m.cons1 = pyo.Constraint(expr = m.d['D'] + m.d['D'] + m.d['Y'] == m.d['N'] + 10 * m.co1) # addition of column 1
m.cons2 = pyo.Constraint(expr = m.co1 + m.d['L'] + m.d['N'] + m.d['C'] == m.d['A'] + 10 * m.co2) # addition of column 2
m.cons3 = pyo.Constraint(expr = m.co2 + m.d['A'] + m.d['A'] + m.d['N'] == m.d['G'] + 10 * m.co3) # addition of column 3
m.cons4 = pyo.Constraint(expr = m.co3 + m.d['N'] + m.d['A'] == m.d['A'] + 10 * m.co4) # addition of column 4
m.cons5 = pyo.Constraint(expr = m.co4 + m.d['O'] + m.d['N'] == m.d['E'] + 10 * m.co5) # addition of column 5
m.cons6 = pyo.Constraint(expr = m.co5 + m.d['R'] == m.d['R']) # addition of column 6
m.cons8 = pyo.Constraint(expr = 100 * m.d['A'] + 10 * m.d['N'] + m.d['D'] == 3 * m.y) # ensures AND is a multiple of 3

m.obj = pyo.Objective(expr = 1, sense = pyo.minimize) # dummy objective

In [42]:
sol = pyo.SolverFactory('glpk')
results = sol.solve(m, tee = True)

    solver failure.
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/h0/0h0h0yrs6f76syrmv1_q13mw0000gn/T/tmpcj0fg_p8.glpk.raw
 --wglp /var/folders/h0/0h0h0yrs6f76syrmv1_q13mw0000gn/T/tmprgsxna50.glpk.glp
 --cpxlp /var/folders/h0/0h0h0yrs6f76syrmv1_q13mw0000gn/T/tmpsgxa482l.pyomo.lp
Reading problem data from '/var/folders/h0/0h0h0yrs6f76syrmv1_q13mw0000gn/T/tmpsgxa482l.pyomo.lp'...
/var/folders/h0/0h0h0yrs6f76syrmv1_q13mw0000gn/T/tmpsgxa482l.pyomo.lp:576: warning: lower bound of variable 'x1' redefined
/var/folders/h0/0h0h0yrs6f76syrmv1_q13mw0000gn/T/tmpsgxa482l.pyomo.lp:576: warning: upper bound of variable 'x1' redefined
38 rows, 117 columns, 329 non-zeros
106 integer variables, 100 of which are binary
676 lines were read
Writing problem data to '/var/folders/h0/0h0h0yrs6f76syrmv1_q13mw0000gn/T/tmprgsxna50.glpk.glp'...
526 lines were written
GLPK Integer Optimizer, v4.65
38 rows, 117 columns, 329 non-zeros
106 integer variables, 100 o

In [45]:
# solution
for c in letters:
    print(c, '-', round(m.d[c]()))

N - 8
C - 2
Y - 4
G - 1
A - 6
R - 3
E - 9
D - 7
L - 5
O - 0


In [51]:
def getnum(m, mystring):
    numstr = ''
    for c in mystring:
        numstr = numstr + str(round(m.d[c]()))
    return numstr

In [56]:
f"{getnum(m, 'RONALD')} + {getnum(m, 'AND')} + {getnum(m, 'NANCY')} = {getnum(m, 'REAGAN')}"

'308657 + 687 + 86824 = 396168'

In [57]:
# check the addition
308657 + 687 + 86824

396168